In [1]:
# --- Fix Imports for Notebook (Explicit Path) ---
import sys
import os

# --- DEFINE YOUR PROJECT ROOT PATH HERE ---
# Manually set the absolute path to your main project folder
project_root = r"F:\Programiranje\Learning_Trading_Bot\Razvijanje_Strategij" 
# --- END DEFINE PATH ---

# --- Add Debug Prints ---
print(f"Explicitly Set Project Root: {project_root}")
if os.path.exists(project_root):
    print(f"Project Root Exists.")
    print(f"Contents of Project Root: {os.listdir(project_root)}") # Check if 'data' is listed
else:
    print(f"❌ ERROR: The specified project_root path does not exist!")
    print("Please double-check the path.")
# --- End Debug Prints ---

# Add the project root to the Python path if it's not already there
if project_root not in sys.path:
    print(f"Adding project root to sys.path...")
    sys.path.append(project_root)
    # --- Add another debug print ---
    print("Current sys.path:")
    import pprint
    pprint.pprint(sys.path)
    # --- End Debug Print ---
else:
    print(f"Project root already in sys.path.")
# --- End Fix ---

print("-" * 20)
print("Attempting imports...")

# Now try the import in the *same cell* or the next one
try:
    import pandas as pd
    import numpy as np
    from data import save_load # <--- This should now definitely work!
    print("✅ Imports successful!")
except ModuleNotFoundError as e:
    print(f"❌ Import Failed: {e}")
    print("Check the 'Explicitly Set Project Root' and its contents above.")

Explicitly Set Project Root: F:\Programiranje\Learning_Trading_Bot\Razvijanje_Strategij
Project Root Exists.
Contents of Project Root: ['.vscode', 'config', 'core', 'data', 'DOCUMENTATION.md', 'experiment_outputs', 'indicators', 'MachineLearning', 'main.py', 'main_v2.py', 'optimization', 'patterns', 'requirements.txt', 'statistical_analysis.py', 'strategies', 'strategy_components', 'test.py', 'tests', 'trading_env', '__pycache__']
Adding project root to sys.path...
Current sys.path:
['C:\\Users\\janju\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip',
 'C:\\Users\\janju\\AppData\\Local\\Programs\\Python\\Python310\\DLLs',
 'C:\\Users\\janju\\AppData\\Local\\Programs\\Python\\Python310\\lib',
 'C:\\Users\\janju\\AppData\\Local\\Programs\\Python\\Python310',
 'f:\\Programiranje\\Learning_Trading_Bot\\Razvijanje_Strategij\\trading_env',
 '',
 'f:\\Programiranje\\Learning_Trading_Bot\\Razvijanje_Strategij\\trading_env\\lib\\site-packages',
 'F:\\Programiranje\\Learning_Trading_B

In [2]:
DATA_PATH = r"F:\Programiranje\Learning_Trading_Bot\Razvijanje_Strategij\data\files\crypto"
DATA_FILENAME = "BTCUSDT_15m.csv"
SAVE_MODEL_ARTIFACT = False


main_df = save_load.load_and_prepare_data(path=DATA_PATH, filename=DATA_FILENAME)


Data loaded and prepared. Shape: (278396, 6)


In [3]:
import ta
import numpy as np


def add_adx_ma(df: pd.DataFrame, adx_period: int = 14, ma_period: int = 20) -> pd.DataFrame:
    # Calculate ADX
    df['adx'] = ta.trend.adx(high=df['High'], low=df['Low'], close=df['Close'], window=adx_period)

    # Calculate Simple Moving Average (SMA)
    df['ma'] = df['Close'].rolling(window=ma_period).mean()

    return df

In [4]:
df = add_adx_ma(main_df, adx_period=14, ma_period=50)
df

,Datetime,Close,Open,High,Low,Volume,adx,ma
0,2017-08-17 04:00:00,4261.48,4261.48,4280.56,4261.48,2.189061,0.000000,NaN
1,2017-08-17 04:15:00,4261.45,4261.48,4270.41,4261.32,9.119865,0.000000,NaN
2,2017-08-17 04:30:00,4310.07,4280.00,4310.07,4267.99,21.923552,0.000000,NaN
3,2017-08-17 04:45:00,4308.83,4310.07,4313.62,4291.37,13.948531,0.000000,NaN
4,2017-08-17 05:00:00,4304.31,4308.83,4328.69,4304.31,5.101153,0.000000,NaN
...,...,...,...,...,...,...,...,...
278391,2025-07-31 23:00:00,116362.99,116010.60,116362.99,115940.02,367.983300,39.653082,117677.3560
278392,2025-07-31 23:15:00,116040.21,116362.98,116362.99,116037.76,132.995530,39.877635,117628.0598
278393,2025-07-31 23:30:00,115743.58,116040.21,116040.21,115500.00,535.601370,41.174297,117572.2286
278394,2025-07-31 23:45:00,115764.08,115743.58,115860.00,115573.45,229.446090,42.378341,117519.1102


In [10]:
adx_treshold = 25
adx_series = df["adx"]
ma_series = df["ma"]
close_series = df["Close"]

conditions = [
    (adx_series >=adx_treshold) & (close_series >= ma_series),
    (adx_series >=adx_treshold) & (close_series <= ma_series)
]
choices = [1, -1]

target_values = np.select(conditions, choices, default=0)
target_name = "target_regime"

target_df = pd.DataFrame({target_name: target_values}, index=df.index)
target_df[target_name] = target_df[target_name].fillna(0).astype(int)

target_names = list(target_df.columns)

target_df.value_counts()

target_regime
 0               153817
-1                65517
 1                59062
Name: count, dtype: int64